In [ ]:
pip install python-mecab-ko

https://docs.google.com/spreadsheets/d/1-9blXKjtjeKZqsf4NzHeYJCrr49-nXeRF6D80udfcwY/edit#gid=589544265 - Mecab pos tagging
https://m.blog.naver.com/zzangdol57/30169103790 - 어미 뜻

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import sys
sys.path.insert(1, '/kaggle/input/kobart-model')
 
train_data_path = os.path.join('/kaggle/input/news-for-chlidd/train')
valid_data_path = os.path.join('/kaggle/input/news-for-chlidd/val')
test_data_path = os.path.join('/kaggle/input/news-for-chlidd/test')

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, DataLoader , Dataset , TensorDataset
from torchmetrics.regression import MeanSquaredError
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

import re
import random

import pandas as pd
from transformers import BartModel
from transformers import DataCollatorWithPadding

from transformers.optimization import AdamW, get_cosine_schedule_with_warmup

from kobart import get_kobart_tokenizer, get_pytorch_kobart_model
from mecab import MeCab

In [ ]:
class News_Dataset(Dataset):
    def __init__(self,root_path , p = 0.5):
        true_path = os.path.join(root_path,str(1))
        false_path = os.path.join(root_path,str(0))

        true_file_list = [os.path.join(true_path,f) for f in os.listdir(true_path)]
        false_file_list = [os.path.join(false_path,f) for f in os.listdir(false_path)][:len(true_file_list)]

        # 최종 파일경로 모음 및 라벨
        self.file_path = true_file_list + false_file_list
        self.label = [1 for _ in range(len(true_file_list))] + [0 for _ in range(len(false_file_list))]

        # 섞어주기
        data = list(zip(self.file_path, self.label))
        random.shuffle(data)
        self.file_path, self.label = zip(*data)
        self.mecab = MeCab()
        self.p = p

    def __len__(self):
        return len(self.label)
    
    def remove_stop_words(self, text, stop_words):
        # stop_words에 포함된 단어들을 정규 표현식으로 만들어 제거
        pattern = '|'.join(stop_words)
        result = re.sub(pattern, '', text)
        return result

    def __getitem__(self, idx):
        with open(self.file_path[idx], 'r') as file:
            title = file.readline().rstrip('\n')
            content = file.readline()
        
        if random.random() < self.p:
            stop_word = [word for word , pos in self.mecab.pos(content) if 'EF' in pos]
            content = self.remove_stop_words(content,stop_word)
    
        return {'x':'<s>'+title+'<unused0>'+content+'</s>' , 'y': torch.tensor([self.label[idx]],dtype=torch.float)}


tokenizer = tokenizer = get_kobart_tokenizer()
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def my_collate_fn(samples):
    '''
        [{'x':["<s>title_1<unused0>content_1</s>"] , 'y' : tensor([y1])},
         {'x':["<s>title_2<unused0>content_2</s>"] , 'y' : tensor([y2])} ... ]
    
    '''

    collate_y = [sample['y'] for sample in samples]
    collate_x = [sample['x'] for sample in samples]

    tokenized_x = data_collator(tokenizer.batch_encode_plus(collate_x))

    return {'input_ids': tokenized_x.input_ids,
            'attention_mask' : tokenized_x.attention_mask,
            'labels': torch.stack(collate_y)}

In [ ]:
train_dataset = News_Dataset(train_data_path)
valid_dataset = News_Dataset(valid_data_path)
test_dataset = News_Dataset(test_data_path)
len(train_dataset) , len(valid_dataset) , len(test_dataset) 

In [ ]:
batch_size = 4
epochs = 1
num_workers = 2

train_loader =  DataLoader(train_dataset,batch_size=batch_size,shuffle=True,collate_fn=my_collate_fn,num_workers=num_workers)
valid_loader = DataLoader(valid_dataset,batch_size=batch_size,shuffle=False,collate_fn=my_collate_fn,num_workers=num_workers)

In [ ]:
class RM_Model(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model_arg = model_arg = {
            'loader_len' : len(train_loader),
            'warmup_ratio' : 0.1,
            'lr' : 5e-5,
            'max_epochs' : epochs,
            'batch_size' : batch_size
        }

        self.bart = BartModel.from_pretrained(get_pytorch_kobart_model())

        self.linear_1 = nn.Linear(768,  768)
        self.dropout = nn.Dropout(0.1)
        self.linear_2 = nn.Linear(768,  1)

        #self.loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([8.1])) 
        self.loss_fn = nn.BCEWithLogitsLoss()
        self.lr = 0.0


    def forward(self, input_ids, attention_mask):
        outputs = self.bart(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )

        hidden_states = outputs.last_hidden_state
        eos_mask = input_ids.eq(1).to(hidden_states.device)
        sentence_representation = hidden_states[eos_mask, :].view(hidden_states.size(0), -1, hidden_states.size(-1))[:, -1, :]
        logits = self.linear_2(self.dropout(self.linear_1(sentence_representation)))
        return logits
    
    def training_step(self, batch, batch_idx):
        logits = self.forward(batch["input_ids"], batch["attention_mask"])
        loss = self.loss_fn(logits, batch["labels"])
        self.log_dict({'loss':loss , 'lr' : self.lr})
        return loss
    
    def validation_step(self, batch, batch_idx):
        logits = self.forward(batch["input_ids"], batch["attention_mask"])
        loss = self.loss_fn(logits, batch["labels"])
        self.log_dict({'val_loss':loss , 'lr' : self.lr})

    def configure_optimizers(self):
        param_optimizer = list(self.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [
                    p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
                ],
                "weight_decay": 0.01,
            },
            {
                "params": [
                    p for n, p in param_optimizer if any(nd in n for nd in no_decay)
                ],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=self.model_arg['lr'], correct_bias=False)
        num_train_steps = int(self.model_arg['loader_len']*self.model_arg['max_epochs'])
        num_warmup_steps = int(num_train_steps * self.model_arg['warmup_ratio'])
        scheduler = get_cosine_schedule_with_warmup(
            optimizer,
            num_warmup_steps=num_warmup_steps,
            num_training_steps=num_train_steps,
        )
        lr_scheduler = {
            "scheduler": scheduler,
            "monitor": "loss",
            "interval": "step",
            "frequency": 1,
        }
        self.lr = optimizer.param_groups[0]['lr']
        return [optimizer], [lr_scheduler]

In [ ]:
logger = pl.loggers.CSVLogger("logs", name="RM_training1")
checkpoint_callback = ModelCheckpoint(
    save_top_k=5,
    monitor="val_loss",
    mode="min",
    filename="RM_Model-{epoch:02d}-{val_loss:.2f}",
)

trainer = pl.Trainer(max_epochs=epochs,logger=logger,accelerator="auto",
                     callbacks=checkpoint_callback,
                     log_every_n_steps=10,
                     )

model = RM_Model()
trainer.fit(model,train_loader,valid_loader)

In [ ]:
model.eval()
sigmoid_fn = nn.Sigmoid()
test_loader = DataLoader(test_dataset,batch_size=1,shuffle=True,collate_fn=my_collate_fn)
for test_data in test_loader:
    logit = model.forward(test_data["input_ids"], test_data["attention_mask"])
    logit = sigmoid_fn(logit)
    print(f"실제 : 모델 => {test_data['labels'].item()} : {logit.item()}")

# 저장 및 로드

In [ ]:
import os
os.chdir(r'/kaggle/working')

from IPython.display import FileLink
FileLink(r'test_checkpoint.ckpt')

In [ ]:
trainer.save_checkpoint("test_checkpoint.ckpt")

In [ ]:
model = RM_Model.load_from_checkpoint("test_checkpoint.ckpt")
model.eval()

In [ ]:
title = "제임스 웹, 모래 비가 내리는 외계행성 발견"
content = """2021년 개봉한 과학영화 ‘듄’은 가상의 모래행성 아라키스를 배경으로 했다. 
스타워즈에 나오는 모래행성 타투인 역시 아라키스에서 모티브를 따왔다.
인류가 만든 최대 규모의 우주망원경이 모래 비가 내리는 영화보다 더 비현실적인 외계행성을 포착했다. 
외계행성의 특이한 대기를 분석하면 우리 태양계의 행성 진화 과정을 더욱 깊게 이해할 수 있을 것으로 기대된다.
벨기에 루뱅대 물리·천문학과의 린 데신(Leen Decin) 교수 연구진은 16일 국제 학술지 ‘네이처’에 “제임스 웹 우주망원경으로 외계행성 WASP-107b의 대기에서 수증기와 이산화황, 규산염 모래 구름을 발견했다”고 밝혔다.
제임스 웹은 미국과 유럽, 캐나다가 25년간 13조원을 들여 개발한 사상 최대 크기의 우주 망원경이다. 
2021년 크리스마스에 우주로 발사돼 이듬해 1월 지구에서 150만㎞ 떨어진 관측 지점에 도착했다. 
이번에 12국 29개 연구 기관에서 천문학자 46명이 참여한 유럽 컨소시엄이 WASP-107b 관측을 진행했다.
WASP-107b는 태양계 밖에 있는 외계행성이다. 2017년 지구로부터서 200광년(光年, 1광년은 빛이 1년 가는 거리로 약 9조4600억㎞) 거리에서 처음 발견됐다. 
태양보다 온도가 낮고 질량이 적은 별(항성)인 WASP-107을 공전한다.
데신 교수가 이끈 국제 공동 연구진은 이른바 투과 분광학이라는 방법으로 외계행성의 대기를 분석했다. 행성이 지나가면 항성에서 나온 빛이 대기를 통과한다. 
대기를 구성하는 물질마다 흡수하는 빛의 파장이 다르다. 
과학자들은 제임스 웹의 중적외선 관측장비(MIRI)로 항성에서 나온 빛의 파장이 외계행성을 지나면서 어떻게 달라지는지 포착해 대기의 구성 성분을 알아냈다.
WASP-107b 행성은 질량이 태양계의 가스 행성인 해왕성과 비슷하지만 크기는 해왕성보다 훨씬 큰 목성과 비슷하다. 
해왕성은 반지름이 지구의 4배이고, 목성은 11배나 된다. 
이 외계행성은 해왕성보다 훨씬 밀도가 낮아 말 그대로 부풀어 있다고 볼 수 있다.
외계행성의 대기 밀도가 낮은 덕분에 연구진은 대기 안쪽까지 관측할 수 있었다. 
그 결과 대기에서 수증기와 이산화황뿐 아니라 규산염 모래 구름까지 발견됐다. 온실가스인 메탄은 없었다. 
연구진은 메탄이 없다는 것은 내부가 따뜻하다는 것을 의미한다고 밝혔다.
성냥 타는 냄새로 알려진 이산화황이 발견된 것도 대기 상태를 짐작게 했다. 
행성이 공전하는 항성은 온도가 낮아 상대적으로 적은 에너지를 방출하지만, 행성의 대기가 부푼 상태여서 에너지가 내부 깊숙한 곳까지 도달할 수 있다. 
덕분에 이산화황을 생성하는 화학반응이 일어날 수 있다고 연구진은 밝혔다.
"""

In [ ]:
model.eval()
tokened_x = tokenizer.batch_encode_plus(["<s>"+title+'<unused0>'+content+'</s>'])
output = model(torch.tensor(tokened_x['input_ids']),torch.tensor(tokened_x['attention_mask']))
output = sigmoid_fn(output)
output

In [ ]:
title = "제임스 웹, 모래 비가 내리는 외계행성 발견"
content = """2021년 개봉한 과학영화 ‘듄’은 가상의 모래행성 아라키스를 배경으로 했어요. 
스타워즈에 나오는 모래행성 타투인 역시 아라키스에서 모티브를 따왔대요. 
인류가 만든 최대 규모의 우주망원경이 모래 비가 내리는 영화보다 더 비현실적인 외계행성을 포착했답니다. 
외계행성의 특이한 대기를 분석하면 우리 태양계의 행성 진화 과정을 더욱 깊게 이해할 수 있을 것으로 기대돼요.
벨기에 루뱅대 물리·천문학과의 린 데신(Leen Decin) 교수 연구진은 16일 국제 학술지 ‘네이처’에 “제임스 웹 우주망원경으로 외계행성 WASP-107b의 대기에서 수증기와 이산화황, 규산염 모래 구름을 발견했다”고 밝혔다.
제임스 웹은 미국과 유럽, 캐나다가 25년간 13조원을 들여 개발한 사상 최대 크기의 우주 망원경이래요. 
2021년 크리스마스에 우주로 발사돼 이듬해 1월 지구에서 150만㎞ 떨어진 관측 지점에 도착했어요. 
이번에 12국 29개 연구 기관에서 천문학자 46명이 참여한 유럽 컨소시엄이 WASP-107b 관측을 진행했어요.
WASP-107b는 태양계 밖에 있는 외계행성이다. 2017년 지구로부터서 200광년(光年, 1광년은 빛이 1년 가는 거리로 약 9조4600억㎞) 거리에서 처음 발견됐어요. 
태양보다 온도가 낮고 질량이 적은 별(항성)인 WASP-107을 공전해요.
데신 교수가 이끈 국제 공동 연구진은 이른바 투과 분광학이라는 방법으로 외계행성의 대기를 분석했다. 행성이 지나가면 항성에서 나온 빛이 대기를 통과한다. 
대기를 구성하는 물질마다 흡수하는 빛의 파장이 다르다. 
과학자들은 제임스 웹의 중적외선 관측장비(MIRI)로 항성에서 나온 빛의 파장이 외계행성을 지나면서 어떻게 달라지는지 포착해 대기의 구성 성분을 알아냈다.
WASP-107b 행성은 질량이 태양계의 가스 행성인 해왕성과 비슷하지만 크기는 해왕성보다 훨씬 큰 목성과 비슷하다. 
해왕성은 반지름이 지구의 4배이고, 목성은 11배나 된다. 
이 외계행성은 해왕성보다 훨씬 밀도가 낮아 말 그대로 부풀어 있다고 볼 수 있다.
외계행성의 대기 밀도가 낮은 덕분에 연구진은 대기 안쪽까지 관측할 수 있었다. 
그 결과 대기에서 수증기와 이산화황뿐 아니라 규산염 모래 구름까지 발견됐다. 온실가스인 메탄은 없었다. 
연구진은 메탄이 없다는 것은 내부가 따뜻하다는 것을 의미한다고 밝혔다.
성냥 타는 냄새로 알려진 이산화황이 발견된 것도 대기 상태를 짐작게 했다. 
행성이 공전하는 항성은 온도가 낮아 상대적으로 적은 에너지를 방출하지만, 행성의 대기가 부푼 상태여서 에너지가 내부 깊숙한 곳까지 도달할 수 있다. 
덕분에 이산화황을 생성하는 화학반응이 일어날 수 있다고 연구진은 밝혔다.
"""

In [ ]:
model.eval()
tokened_x = tokenizer.batch_encode_plus(["<s>"+title+'<unused0>'+content+'</s>'])
output = model(torch.tensor(tokened_x['input_ids']),torch.tensor(tokened_x['attention_mask']))
output = sigmoid_fn(output)
output

In [ ]:
title = "더내고 더받는' 포함 국민연금안 제시…소득대체율 40∼50%"
content = """(서울=연합뉴스) 이유미 기자 = 국회 연금개혁특별위원회(연금특위) 산하 민간자문위원회가 국민연금 보험료율을 현행보다 4∼6%포인트(p) 높이고, 소득대체율을 40% 또는 50%로 조정하는 개혁안을 마련했다.
앞서 정부가 마련한 국민연금 종합운영계획안에는 보험료율 인상의 불가피성만 강조한 채 구체적인 인상률 등 모수(숫자)가 빠진 상황에서 특위 차원에서 모수개혁안이 나온 점이 주목된다.
15일 국회 연금특위 민간자문위원회가 특위에 제출한 최종 활동보고서에 따르면 자문위는 '보험료율 13%와 소득대체율 50%', 그리고 '보험료율 15%와 소득대체율 40%' 등 2가지 모수개혁안을 제안했다.
현행 국민연금은 보험료율이 9%이고, 소득대체율은 42.5%이다. 첫번째 안은 보험료율을 현행보다 4%p 높이는 대신, 소득대체율도 7.5%p 올리는 내용이다. 두번째 안은 보험료율을 6%p 높이는 반면, 소득대체율은 2.5%p 낮추는 것이다.
국민연금 기금이 2055년 바닥날 것으로 예상되면서 그동안 보험료율 인상 필요성이 제기돼왔다. 국민연금의 보장성이 낮은 만큼 노후 소득으로서 실효성을 높여야 한다는 주장도 함께 나왔다.자문위의 안은 보험료율 인상 필요성을 반영한 것이다. 
월소득 300만원을 가정할 경우 보험료가 현행 27만원에서 39만∼45만원으로 늘어난다.
자문위는 "국민연금 소득 보장을 강화하자는 입장에서는 공적연금의 장기적 재정 부담이 부담 가능한 수준에 있으므로 공적연금의 정책 목표에 충실할 것을 주장하며 보험료율 인상(13%)과 소득대체율 인상(50%)을 동시에 추진해 소득보장과 재정안정의 균형을 달성하자는 개혁안을 제시했다"고 밝혔다.
또 "재정안정을 중요시하는 입장에서는 국민연금의 재정 불안정을 감안해 소득대체율 인상이 아닌 보험료율(최소12∼15%) 인상이 필요하다는 점을 강조했고, 기초연금 지급 범위도 국민연금의 성숙을 고려해 축소해나갈 필요가 있다고 주장했다"고 보고했다.
자문위는 그러면서 "구조개혁의 큰 틀에 저해되지 않는 선에서 모수 개혁을 우선 추진해 연금 개혁의 지속적 동력을 확보해야 한다"고 강조했다.연금특위는 16일 전체회의에서 특위 보고서와 함께 정부의 연금개혁안을 보고받고 주요 쟁점별 논의를 본격화할 예정이다.
"""

In [ ]:
model.eval()
tokened_x = tokenizer.batch_encode_plus(["<s>"+title+'<unused0>'+content+'</s>'])
output = model(torch.tensor(tokened_x['input_ids']),torch.tensor(tokened_x['attention_mask']))
output = sigmoid_fn(output)
output

In [ ]:
title = "더내고 더받는' 포함 국민연금안 제시…소득대체율 40∼50%"
content = """이유미 기자 = 국회 연금개혁특별위원회(연금특위) 산하 민간자문위원회가 국민연금 보험료율을 현행보다 4∼6%포인트(p) 높이고, 소득대체율을 40% 또는 50%로 조정하는 개혁안을 마련했어요.
앞서 정부가 마련한 국민연금 종합운영계획안에는 보험료율 인상의 불가피성만 강조한 채 구체적인 인상률 등 모수(숫자)가 빠진 상황에서 특위 차원에서 모수개혁안이 나온 점이 주목된대요.
15일 국회 연금특위 민간자문위원회가 특위에 제출한 최종 활동보고서에 따르면 자문위는 '보험료율 13%와 소득대체율 50%', 그리고 '보험료율 15%와 소득대체율 40%' 등 2가지 모수개혁안을 제안했답니다.
현행 국민연금은 보험료율이 9%이고, 소득대체율은 42.5%이래요. 첫번째 안은 보험료율을 현행보다 4%p 높이는 대신, 소득대체율도 7.5%p 올리는 내용이에요. 두번째 안은 보험료율을 6%p 높이는 반면, 소득대체율은 2.5%p 낮추는 것이에요.
국민연금 기금이 2055년 바닥날 것으로 예상되면서 그동안 보험료율 인상 필요성이 제기돼왔어요. 국민연금의 보장성이 낮은 만큼 노후 소득으로서 실효성을 높여야 한다는 주장도 함께 나왔다.자문위의 안은 보험료율 인상 필요성을 반영한 것이랍니다. 
월소득 300만원을 가정할 경우 보험료가 현행 27만원에서 39만∼45만원으로 늘어나요.
자문위는 "국민연금 소득 보장을 강화하자는 입장에서는 공적연금의 장기적 재정 부담이 부담 가능한 수준에 있으므로 공적연금의 정책 목표에 충실할 것을 주장하며 보험료율 인상(13%)과 소득대체율 인상(50%)을 동시에 추진해 소득보장과 재정안정의 균형을 달성하자는 개혁안을 제시했다"고 밝혔어요.
또 "재정안정을 중요시하는 입장에서는 국민연금의 재정 불안정을 감안해 소득대체율 인상이 아닌 보험료율(최소12∼15%) 인상이 필요하다는 점을 강조했고, 기초연금 지급 범위도 국민연금의 성숙을 고려해 축소해나갈 필요가 있다고 주장했다"고 보고했어요.
자문위는 그러면서 "구조개혁의 큰 틀에 저해되지 않는 선에서 모수 개혁을 우선 추진해 연금 개혁의 지속적 동력을 확보해야 한다"고 강조했어요.연금특위는 16일 전체회의에서 특위 보고서와 함께 정부의 연금개혁안을 보고받고 주요 쟁점별 논의를 본격화할 예정이에요.
"""

In [ ]:
model.eval()
tokened_x = tokenizer.batch_encode_plus(["<s>"+title+'<unused0>'+content+'</s>'])
output = model(torch.tensor(tokened_x['input_ids']),torch.tensor(tokened_x['attention_mask']))
output = sigmoid_fn(output)
output

11.17 - 20시